In [1]:
import numpy as np
import ROOT
import os

In [2]:
ROOT.gInterpreter.AddIncludePath("../include")

In [ ]:
ROOT.gInterpreter.Declare(
"""
#include <ROOT/RVec.hxx>
#include "CustomContainers.hpp"
"""
)

In [4]:
ROOT.gStyle.SetOptFit()

In [5]:
loadedLib = ROOT.gSystem.Load("../lib/libCustomContainers.so")

# Unsuccessful library load.
if loadedLib < 0:
    raise ImportError(f"System flag is: {loadedLib}")

In [6]:
os.chdir("../build")

In [7]:
simulationFile = ROOT.TFile("SimulationResults.root")

In [ ]:
for key in simulationFile.GetListOfKeys():
    print(key.GetName())

# Fe-55 Spectrum

In [9]:
specHist = simulationFile.Get("Electron-ion pairs")

In [ ]:
specCanvas = ROOT.TCanvas()

specHist.Draw()
specCanvas.Draw()

# Detector properties

In [11]:
driftDataframe = ROOT.RDataFrame("DriftLines", "SimulationResults.root")

In [ ]:
driftDataframe.Describe()

In [ ]:
atsArr = driftDataframe.AsNumpy(["ArrivalTimeSpread"])["ArrivalTimeSpread"]

atsQuantiles = np.quantile(atsArr, [0.25, 0.75])
atsIQR = atsQuantiles[1] - atsQuantiles[0]

bin_edges = np.histogram_bin_edges(atsArr, bins="fd", range=(atsQuantiles[0] - 1.5 * atsIQR, atsQuantiles[1] + 1.5 * atsIQR))
n_bins = bin_edges.size - 1

atsHist = driftDataframe.Histo1D(("Arrival Time Spread [ns]", "Arrival Time Spread [ns]", n_bins, bin_edges), "ArrivalTimeSpread")

atsCanvas = ROOT.TCanvas()

atsHist.Draw()
atsCanvas.Draw()

In [ ]:
gainArr = driftDataframe.AsNumpy(["Gain"])["Gain"]

gainQuantiles = np.quantile(gainArr, [0.25, 0.75])
gainIQR = gainQuantiles[1] - gainQuantiles[0]

bin_edges = np.histogram_bin_edges(gainArr, bins="fd", range=(gainQuantiles[0] - 1.5 * gainIQR, gainQuantiles[1] + 1.5 * gainIQR))
n_bins = bin_edges.size - 1

gainHist = driftDataframe.Histo1D(("Gain", "Gain", n_bins, bin_edges), "Gain")

gainCanvas = ROOT.TCanvas()

gainHist.Draw()
gainCanvas.Draw()

In [ ]:
lossArr = driftDataframe.AsNumpy(["Loss"])["Loss"]

lossQuantiles = np.quantile(lossArr, [0.25, 0.75])
lossIQR = lossQuantiles[1] - lossQuantiles[0]

bin_edges = np.histogram_bin_edges(lossArr, bins="fd", range=(lossQuantiles[0] - 1.5 * lossIQR, lossQuantiles[1] + 1.5 * lossIQR))
n_bins = bin_edges.size - 1

lossHist = driftDataframe.Histo1D(("Loss", "Loss", n_bins, bin_edges), "Loss")

lossHist.Fit("landau", "ME")

lossCanvas = ROOT.TCanvas()

lossHist.Draw()
lossCanvas.Draw()

In [ ]:
eleArr = driftDataframe.AsNumpy(["SizeElectrons"])["SizeElectrons"]

eleQuantiles = np.quantile(eleArr, [0.25, 0.75])
eleIQR = eleQuantiles[1] - eleQuantiles[0]

bin_edges = np.histogram_bin_edges(eleArr, bins="fd", range=(eleQuantiles[0] - 1.5 * eleIQR, eleQuantiles[1] + 1.5 * eleIQR))
n_bins = bin_edges.size - 1

nEleHist = driftDataframe.Histo1D(("Avalanche Electrons", "Avalanche Electrons", n_bins, bin_edges), "SizeElectrons")

nEleHist.Fit("gaus", "ME")

nEleCanvas = ROOT.TCanvas()

nEleHist.Draw()
nEleCanvas.Draw()

In [ ]:
ionArr = driftDataframe.AsNumpy(["SizeIons"])["SizeIons"]

ionQuantiles = np.quantile(ionArr, [0.25, 0.75])
ionIQR = ionQuantiles[1] - ionQuantiles[0]

bin_edges = np.histogram_bin_edges(ionArr, bins="fd", range=(ionQuantiles[0] - 1.5 * ionIQR, ionQuantiles[1] + 1.5 * ionIQR))
n_bins = bin_edges.size - 1

nIonHist = driftDataframe.Histo1D(("Avalanche Ions", "Avalanche Ions", n_bins, bin_edges), "SizeIons")

nIonHist.Fit("gaus", "ME")

nIonCanvas = ROOT.TCanvas()

nIonHist.Draw()
nIonCanvas.Draw()